In [1]:
import os
import pandas as pd
from collections import defaultdict

from IPython.display import HTML
from IPython.display import display_html

In [2]:
TPCDS_FN = "tpcds_queries_df.csv"
JOB_FN = "job_queries_df.csv"
CEB_FN = "ceb_queries_df.csv"

In [3]:
tpdf = pd.read_csv(TPCDS_FN)
jobdf = pd.read_csv(JOB_FN)
cebdf = pd.read_csv(CEB_FN)
tpdf["dataset"] = "TPCDS"
jobdf["dataset"] = "JOB"
cebdf["dataset"] = "CEB"

In [4]:
print(len(tpdf))
print(len(jobdf))
print(len(cebdf))

506
706
25939


In [5]:
df = pd.concat([tpdf, jobdf, cebdf])

In [6]:
print(df.keys())

Index(['left', 'table', 'col', 'op', 'val', 'qid', 'dtype', 'dkind',
       'likedtype', 'likekind', 'discrete_ops', 'in_ops', 'cont_ops',
       'like_ops', 'dataset'],
      dtype='object')


In [7]:
# HTML(df[["like_ops", "discrete_ops", "cont_ops"]].\
#      describe(percentiles=[0.9,0.99]).reset_index().to_html(index=False))

In [8]:
def pretty_print(df):
    return display( HTML( df.to_html(index=False).replace("\\n","<br>") ) )

In [10]:
STATCOLS = ["like_ops", "discrete_ops", "cont_ops"]
STATSTODROP = ["std", "min", "max", "25%", "50%","75%"]
curdrops = []

for si, stc in enumerate(STATCOLS):
    for sdrop in STATSTODROP:
        curdrops.append(tuple([stc, sdrop]))
    if si != 0:
        curdrops.append(tuple([stc,"count"]))

#cdf = df[[""]]
cdf = df[["dataset","like_ops", "cont_ops", "discrete_ops", "in_ops"]]

pretty_print(cdf.groupby(["dataset"])[STATCOLS].describe().\
     reset_index().\
     drop(columns=curdrops).\
     rename(columns={"like_ops":"LIKE", "discrete_ops":"Discrete", "cont_ops":"Continuous",
                    "mean":"Fraction"}).\
     round(3))

In [11]:
discdf = df[df.discrete_ops == 1]

tmp = discdf.groupby(["dataset","dkind"]).count()["discrete_ops"]
tmp = tmp.groupby(level=0).apply(lambda x: 100 * x / float(x.sum()))

# pretty_print(tmp.reset_index()\
#              .rename(columns={"discrete_ops":"Percentage"})
#             )
tmp = tmp.reset_index().set_index(['dataset', 'dkind']).rename(
    columns={"discrete_ops":"Percentage"})
tmp

Percentage
dataset dkind            
CEB     eq       7.343486
        in      92.656514
JOB     eq      68.433180
        in      26.036866
        neq      5.529954
TPCDS   eq      77.922078
        in      22.077922

In [12]:
tmp = discdf.groupby(["dataset","dtype"]).count()["discrete_ops"]
tmp = tmp.groupby(level=0).apply(lambda x: 100 * x / float(x.sum()))

# pretty_print(tmp.reset_index()\
#              .rename(columns={"discrete_ops":"Percentage"})
#             )

tmp = tmp.reset_index().set_index(['dataset', 'dtype']).rename(
    columns={"discrete_ops":"Percentage"})
tmp

Percentage
dataset dtype             
CEB     date      0.129724
        string   99.870276
JOB     date      5.299539
        string   94.700461
TPCDS   date     26.623377
        num      23.051948
        sql       4.545455
        string   45.779221

In [13]:
# tmp = discdf.groupby(["dataset","dtype"]).count()["discrete_ops"]
# tmp = tmp.groupby(level=0).apply(lambda x: 100 * x / float(x.sum()))
# tmp.reset_index()

In [14]:
#ldf["val"]
def get_len(val):
    val = val.replace("%", "")
    val = val.replace("'", "")
    return len(val)

In [15]:
ldf = df[df.like_ops == 1]
#ldf.groupby("likekind").count()["discrete_ops"].reset_index().sort_values(by="discrete_ops", ascending=False)

ldf["Filter Length"] = ldf.apply(lambda x: get_len(x["val"]), axis=1)
tmp = ldf.groupby(["dataset","likekind"]).count()["discrete_ops"]
tmp = tmp.groupby(level=0).apply(lambda x: 100 * x / float(x.sum()))


tmp = tmp.reset_index().set_index(['dataset', 'likekind']).rename(
    columns={"discrete_ops":"Percentage"})
tmp

<ipython-input-15-eaa726996ab5>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ldf["Filter Length"] = ldf.apply(lambda x: get_len(x["val"]), axis=1)


Percentage
dataset likekind            
CEB     contains  100.000000
JOB     3%         11.363636
        4%          0.568182
        contains   75.568182
        starts     12.500000
TPCDS   starts    100.000000

In [16]:
tmp = ldf.groupby(["dataset","likekind"])[["Filter Length"]].describe(percentiles=[0.9])
#tmp = tmp.groupby(level=0).apply(lambda x: 100 * x / float(x.sum()))
DROPS = ["std", "min", "max"]
dc = []
for dr in DROPS:
    dc.append(("Filter Length", dr))

tmp = tmp.reset_index().set_index(['dataset', 'likekind']).rename(
    columns={}).drop(columns=dc)
tmp

Filter Length                       
                         count       mean   50%   90%
dataset likekind                                     
CEB     contains        1778.0   2.347019   2.0   4.0
JOB     3%                20.0   7.700000   6.0  12.0
        4%                 1.0  11.000000  11.0  11.0
        contains         133.0   6.300752   6.0   9.8
        starts            22.0   6.727273   7.5  13.0
TPCDS   starts             1.0   5.000000   5.0   5.0

In [17]:
#ldf = df[df.like_ops == 1]

#ldf["likedtype"] = ldf.apply(lambda x: if x["likedtype"] == -1: then ,axis=1)

ldtypes = []
for idx,row in ldf.iterrows():
    if row["likedtype"] == "-1" or row["likedtype"] == -1 or row["likedtype"] == "-1.0":
        ldtypes.append("unknown")
    else:
        ldtypes.append(row["likedtype"])
ldf["likedtype"] = ldtypes

tmp = ldf.groupby(["dataset","likedtype"]).count()["discrete_ops"]
tmp = tmp.groupby(level=0).apply(lambda x: 100 * x / float(x.sum()))
tmp = tmp.reset_index().set_index(['dataset', 'likedtype']).rename(
    columns={"discrete_ops":"Percentage"})
tmp

<ipython-input-17-af36808a2fd5>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ldf["likedtype"] = ldtypes


Percentage
dataset likedtype            
CEB     num         14.623172
        short       51.012373
        unknown     23.509561
        word        10.686164
        words        0.168729
JOB     short       10.795455
        unknown     42.045455
        word        40.909091
        words        6.250000
TPCDS   unknown    100.000000

In [18]:
#inpdata.groupby("col").count().reset_index()["val"].describe(percentiles=[0.75,0.9,0.99])

In [19]:
allinpdata = defaultdict(list)
for idx, row in df.iterrows():
    op = row["op"]
    if op in ["=", "!="]:
        allinpdata["col"].append(row["col"])
        allinpdata["val"].append(row["val"])
        allinpdata["dataset"].append(row["dataset"])

    elif op == "in":
        if "select" in row["val"]:
            pass
        else:
            cvals = row["val"]
            cvals = cvals.replace("(", "")
            cvals = cvals.replace(")", "")
            cvals = cvals.split(",")
            for cval in cvals:
                allinpdata["dataset"].append(row["dataset"])
                allinpdata["col"].append(row["col"])
                allinpdata["val"].append(cval)

In [20]:
inpdata = pd.DataFrame(allinpdata)
print(len(inpdata))
inpdata = inpdata.drop_duplicates()
inpdata = inpdata[inpdata["col"] != "function"]
print(len(inpdata))
print(len(set(inpdata["col"])))
print(set(inpdata["col"]))

68621
7768
76
{nan, 'd_dom', 'syear', 'year', 'i_item_id', 'c_preferred_cust_flag', 'ca_zip', 'i_manager_id', 'ss_item_sk', 'i_class', 'i_item_sk', 'gender', 'surname_pcode', 'dyear', 'i_brand', 'hd_dep_count', 'info', 's_state', 'i_manufact_id', 'p_channel_event', 'd_date', 'i_size', 'name_pcode_nf', 'cd_education_status', 'c_birth_country', 'r_reason_desc', 'i_units', 'year_total', 'name', 'd_month_seq', 'd_qoy', 'hd_buy_potential', 'd_year', 'd_moy', 'c_birth_month', 's_store_name', 'web_company_name', 'title', 'note', 'cs_qty', 'id', 'ws_item_sk', 'p_channel_dmail', 'ws_net_profit', 's_market_id', 'ca_gmt_offset', 'd_dow', 'cc_county', 'ca_state', 'link', 'i_color', 't_hour', 'p_channel_email', 'country_code', 'ca_country', 'd_quarter_name', 'cd_gender', 'd_week_seq', 'keyword', 'ss_net_profit', 'role', 'p_channel_tv', 'ca_city', 'production_year', 'name_pcode_cf', 'ss_store_sk', 'i_category', 'cd_marital_status', 'hd_vehicle_count', 's_city', 'sale_type', 'sm_carrier', 's_county',

In [21]:
inpdata.groupby("col").count().reset_index()["val"].describe(percentiles=[0.75,0.9,0.99])

count      75.000000
mean      102.813333
std       415.657853
min         1.000000
50%         3.000000
75%        11.000000
90%       139.200000
99%      1829.080000
max      3056.000000
Name: val, dtype: float64

In [22]:
STATCOLS = ["like_ops", "discrete_ops", "cont_ops"]
STATSTODROP = ["std", "min", "max", "25%", "50%","75%"]
curdrops = []

pretty_print(
    inpdata.groupby(["dataset", "col"]).nunique().reset_index()[["dataset","val"]]\
    .groupby(["dataset"])[["val"]].describe(percentiles=[0.75,0.9,0.99]).reset_index()\
    .rename(columns={"count":"Total \\n Columns", "val":"Number of unique constants"})\
    .round(3)
)